# Adaptive Fermi-Hubbard Simulation with Maestro

This notebook demonstrates a **3-tier adaptive simulation pipeline** for the 1D Fermi-Hubbard model — a fundamental model of strongly correlated electrons.

The key insight: for **local quench dynamics**, information propagates at a finite speed (**Lieb-Robinson velocity**). Most of the system remains frozen, so we can simulate a 200-qubit system by solving only the ~40-qubit active region.

### Pipeline

| Tier | Backend | Purpose | Cost |
|------|---------|---------|------|
| 1. Scout | Pauli Propagator | Detect the active light cone | O(n), seconds |
| 2. Sniper | MPS CPU (χ=64) | Quick physics preview | O(n_active · χ³) |
| 3. Precision | MPS GPU (χ=256) | Converged results | O(n_active · χ³), GPU-accelerated |

---

## Physics Background

The **Fermi-Hubbard model** describes electrons hopping on a lattice with on-site repulsion:

$$H = -t \sum_{\langle i,j \rangle, \sigma} (c^\dagger_{i,\sigma} c_{j,\sigma} + \text{h.c.}) + U \sum_i n_{i,\uparrow} n_{i,\downarrow}$$

- **$t$** (hopping): kinetic energy — electrons tunnel between adjacent sites
- **$U$** (interaction): potential energy — penalty for double occupancy
- **$U/t$** ratio: small → metallic, large → Mott insulator

### Jordan-Wigner Mapping

We map fermions to qubits:
- Qubits `[0, N)` → spin-up electrons
- Qubits `[N, 2N)` → spin-down electrons

For 1D nearest-neighbor hopping, no Jordan-Wigner strings are needed:
- **Hopping:** $\exp(-i \, dt \, t \, (XX + YY) / 2)$ per bond
- **Interaction:** $\exp(-i \, dt \, U/4 \, (I - Z_\uparrow - Z_\downarrow + Z_\uparrow Z_\downarrow))$ per site

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

import maestro
from maestro.circuits import QuantumCircuit

%matplotlib inline
plt.rcParams['figure.dpi'] = 120

## Step 1: Building the Fermi-Hubbard Model

The `FermiHubbardModel` class encapsulates the physics. It builds:
1. **Trotterized circuits** for time evolution (with non-Clifford Rz gates)
2. **Clifford-only scout circuits** for Pauli Propagator (using H, CX, CZ)

In [ ]:
class FermiHubbardModel:
    """
    1D Fermi-Hubbard model with Jordan-Wigner mapping.
    """

    def __init__(self, n_sites, t=1.0, u=1.0):
        self.n_sites = n_sites
        self.t = t
        self.u = u

    def build_circuit(self, steps, dt, init_wall_idx, active_sites_range=None):
        """Build Trotterized time-evolution circuit."""
        circuit = QuantumCircuit()

        # State preparation: domain wall
        if active_sites_range:
            start_site, end_site = active_sites_range
            n_active = end_site - start_site
            for local_i in range(n_active):
                if local_i + start_site < init_wall_idx:
                    circuit.x(local_i)
                    circuit.x(local_i + n_active)
        else:
            for i in range(init_wall_idx):
                circuit.x(i)
                circuit.x(i + self.n_sites)

        n_active = (self.n_sites if not active_sites_range
                    else active_sites_range[1] - active_sites_range[0])
        up_offset = 0
        down_offset = n_active

        for _ in range(steps):
            for i in range(0, n_active - 1, 2):
                self._add_hopping(circuit, up_offset + i, up_offset + i + 1, dt)
                self._add_hopping(circuit, down_offset + i, down_offset + i + 1, dt)
            for i in range(1, n_active - 1, 2):
                self._add_hopping(circuit, up_offset + i, up_offset + i + 1, dt)
                self._add_hopping(circuit, down_offset + i, down_offset + i + 1, dt)
            for i in range(n_active):
                self._add_interaction(circuit, up_offset + i, down_offset + i, dt)
        return circuit

    def build_clifford_scout_circuit(self, steps, init_wall_idx):
        """Clifford-only circuit for PP light-cone detection."""
        circuit = QuantumCircuit()
        for i in range(init_wall_idx):
            circuit.x(i)
            circuit.x(i + self.n_sites)

        for step in range(steps):
            lc_start = max(0, init_wall_idx - step - 1)
            lc_end = min(self.n_sites, init_wall_idx + step + 2)
            for i in range(max(0, lc_start), lc_end - 1, 2):
                circuit.h(i); circuit.cx(i, i + 1); circuit.h(i)
                circuit.h(self.n_sites + i)
                circuit.cx(self.n_sites + i, self.n_sites + i + 1)
                circuit.h(self.n_sites + i)
            for i in range(max(1, lc_start | 1), lc_end - 1, 2):
                circuit.cx(i, i + 1)
                circuit.cx(self.n_sites + i, self.n_sites + i + 1)
            for i in range(lc_start, lc_end):
                circuit.cz(i, self.n_sites + i)
        return circuit

    def _add_hopping(self, qc, q1, q2, dt):
        """exp(-iθ(XX + YY)/2) where θ = t·dt."""
        theta = self.t * dt
        qc.h(q1); qc.h(q2)
        qc.cx(q1, q2); qc.rz(q2, theta); qc.cx(q1, q2)
        qc.h(q1); qc.h(q2)
        qc.sdg(q1); qc.sdg(q2)
        qc.h(q1); qc.h(q2)
        qc.cx(q1, q2); qc.rz(q2, theta); qc.cx(q1, q2)
        qc.h(q1); qc.h(q2)
        qc.s(q1); qc.s(q2)

    def _add_interaction(self, qc, q_up, q_down, dt):
        """exp(-i dt U n↑ n↓)."""
        angle = self.u * dt / 4.0
        qc.rz(q_up, -2.0 * angle)
        qc.rz(q_down, -2.0 * angle)
        qc.cx(q_up, q_down)
        qc.rz(q_down, 2.0 * angle)
        qc.cx(q_up, q_down)

print("FermiHubbardModel defined.")

## Step 2: Helper Functions

In [ ]:
def build_z_observables(n_qubits):
    """Build per-qubit Z observables."""
    obs = []
    for i in range(n_qubits):
        pauli = ['I'] * n_qubits
        pauli[i] = 'Z'
        obs.append("".join(pauli))
    return obs


def z_to_density(exp_vals, n_active_sites):
    """Convert ⟨Z⟩ to particle density: n_σ = (1 - ⟨Z⟩)/2, total = n↑ + n↓."""
    densities = []
    for i in range(n_active_sites):
        n_up = (1.0 - exp_vals[i]) / 2.0
        n_down = (1.0 - exp_vals[i + n_active_sites]) / 2.0
        densities.append(n_up + n_down)
    return densities

# Configuration
T_HOP = 1.0
U_INT = 1.0
T_EVOLUTION = 5.0
N_STEPS = 50
CHI_CPU = 64
CHI_GPU = 256
SAFETY_MARGIN = 5

print(f"Parameters: t={T_HOP}, U={U_INT} (U/t={U_INT/T_HOP:.1f}), T={T_EVOLUTION}, {N_STEPS} steps")

## Step 3: Tier 1 — PP Scout (Light-Cone Detection)

The **Pauli Propagator** runs a Clifford-only proxy circuit on the **entire** system. It measures $\langle Z_i \rangle$ for all sites and identifies where they deviate from the initial domain-wall state.

Sites that are "disturbed" by the quench dynamics are within the **Lieb-Robinson light cone** — only these need to be simulated at higher accuracy.

> The Clifford scout uses H, CX, CZ gates within an expanding window around the domain wall. This mimics the causal light cone without saturating the chain.

In [ ]:
# System size
TOTAL_QUBITS = 50    # 25 sites
N_SITES = TOTAL_QUBITS // 2
WALL = N_SITES // 2  # Domain wall at the center

print(f"System: {TOTAL_QUBITS} qubits ({N_SITES} sites)")
print(f"Domain wall at site {WALL}")
print(f"Initial state: |1111...0000...⟩ (left filled, right empty)\n")

# Run PP Scout
light_cone_radius = 2.0 * T_HOP * T_EVOLUTION
scout_steps = max(2, int(light_cone_radius / 2) + 1)

model = FermiHubbardModel(N_SITES, t=T_HOP, u=U_INT)
scout_circuit = model.build_clifford_scout_circuit(
    steps=scout_steps, init_wall_idx=WALL
)
obs_list = build_z_observables(TOTAL_QUBITS)

print(f"Running PP Scout ({scout_steps} Clifford steps)...")
t0 = time.time()
scout_result = scout_circuit.estimate(
    observables=obs_list,
    simulator_type=maestro.SimulatorType.QCSim,
    simulation_type=maestro.SimulationType.PauliPropagator,
)
scout_time = time.time() - t0

z_vals = scout_result['expectation_values']
print(f"Completed in {scout_time:.3f}s")

# Detect active sites
THRESHOLD = 0.001
active_start = WALL
active_end = WALL

for i in range(N_SITES):
    initial_z = -1.0 if i < WALL else 1.0
    if (abs(z_vals[i] - initial_z) > THRESHOLD or
            abs(z_vals[N_SITES + i] - initial_z) > THRESHOLD):
        active_start = min(active_start, i)
        active_end = max(active_end, i + 1)

active_start = max(0, active_start - SAFETY_MARGIN)
active_end = min(N_SITES, active_end + SAFETY_MARGIN)
n_active = active_end - active_start

print(f"\nActive region: sites [{active_start}, {active_end})")
print(f"Active qubits: {2 * n_active} out of {TOTAL_QUBITS} ({TOTAL_QUBITS // (2*n_active)}× reduction)")

In [ ]:
# Visualize the scout results
fig, ax = plt.subplots(figsize=(12, 4))

up_z = z_vals[:N_SITES]
down_z = z_vals[N_SITES:]
sites = list(range(N_SITES))

ax.bar([s - 0.15 for s in sites], up_z, width=0.3, color='#3498DB', alpha=0.8, label='Spin-up ⟨Z⟩')
ax.bar([s + 0.15 for s in sites], down_z, width=0.3, color='#E74C3C', alpha=0.8, label='Spin-down ⟨Z⟩')

# Initial state reference
init_z = [-1.0 if i < WALL else 1.0 for i in range(N_SITES)]
ax.step(sites, init_z, color='gray', linewidth=2, linestyle='--', label='Initial ⟨Z⟩', where='mid')

# Active region
ax.axvspan(active_start - 0.5, active_end - 0.5, alpha=0.1, color='green', label='Active region')
ax.axvline(x=WALL - 0.5, color='red', linestyle=':', linewidth=2, label=f'Domain wall (site {WALL})')

ax.set_xlabel('Site Index', fontsize=12)
ax.set_ylabel('⟨Z⟩', fontsize=12)
ax.set_title(f'Tier 1: PP Scout — {TOTAL_QUBITS} qubits, {scout_time*1000:.0f}ms', fontsize=13)
ax.legend(fontsize=9, loc='upper right')
ax.set_ylim(-1.3, 1.3)
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Sites where ⟨Z⟩ deviates from initial: the light cone around the domain wall")
print(f"Only these sites need to be simulated with MPS!")

## Step 4: Tier 2 — MPS Sniper (CPU, χ=64)

Now we run the **real** Trotterized circuit (with non-Clifford Rz gates) but **only on the active subregion** detected by the scout.

For a 50-qubit system with ~30 active qubits, this saves a factor of ~2×. For a 1000-qubit system with the same light cone, the savings would be ~30×.

In [ ]:
# Run MPS on the active subregion
n_qubits_active = 2 * n_active
print(f"Tier 2: MPS CPU on {n_qubits_active} qubits (χ={CHI_CPU})")
print(f"Active sites: [{active_start}, {active_end}), {n_active} sites")

sniper_model = FermiHubbardModel(n_active, t=T_HOP, u=U_INT)
sniper_circuit = sniper_model.build_circuit(
    steps=N_STEPS, dt=T_EVOLUTION / N_STEPS,
    init_wall_idx=WALL, active_sites_range=(active_start, active_end)
)
sniper_obs = build_z_observables(n_qubits_active)

t0 = time.time()
sniper_result = sniper_circuit.estimate(
    observables=sniper_obs,
    simulator_type=maestro.SimulatorType.QCSim,
    simulation_type=maestro.SimulationType.MatrixProductState,
    max_bond_dimension=CHI_CPU,
)
sniper_time = time.time() - t0

cpu_density = z_to_density(sniper_result['expectation_values'], n_active)
print(f"Completed in {sniper_time:.2f}s")

## Step 5: Tier 3 — Precision (Higher χ)

For converged results, we increase the bond dimension. GPU acceleration would provide 10–100× speedup for the $O(\chi^3)$ tensor contractions.

> To run with GPU: use `SimulatorType.Gpu` instead of `SimulatorType.QCSim`

In [ ]:
# Tier 3: Higher bond dimension (CPU fallback if no GPU)
chi_precision = CHI_GPU
print(f"Tier 3: Precision MPS on {n_qubits_active} qubits (χ={chi_precision})")

precision_circuit = sniper_model.build_circuit(
    steps=N_STEPS, dt=T_EVOLUTION / N_STEPS,
    init_wall_idx=WALL, active_sites_range=(active_start, active_end)
)

t0 = time.time()
precision_result = precision_circuit.estimate(
    observables=sniper_obs,
    simulator_type=maestro.SimulatorType.QCSim,  # Change to .Gpu for GPU
    simulation_type=maestro.SimulationType.MatrixProductState,
    max_bond_dimension=chi_precision,
)
precision_time = time.time() - t0

gpu_density = z_to_density(precision_result['expectation_values'], n_active)
print(f"Completed in {precision_time:.2f}s")

## Step 6: Visualize the Density Profile

The density profile shows how the domain wall spreads over time. The left half starts at density 2 (both spins occupied) and the right at 0 (empty). Charge transport smooths out the step.

In [ ]:
fig, ax = plt.subplots(figsize=(13, 5))

global_x = list(range(active_start, active_end))

# CPU result
ax.bar(global_x, cpu_density, color='#2F847C', width=1.0, alpha=0.7,
       label=f'MPS CPU (χ={CHI_CPU}, {sniper_time:.1f}s)')

# Precision result (overlay)
ax.step([x + 0.5 for x in global_x], gpu_density, color='#E74C3C', linewidth=2,
        where='mid', label=f'MPS Precision (χ={chi_precision}, {precision_time:.1f}s)')

# Frozen regions
if active_start > 0:
    ax.bar(range(0, active_start), [2.0] * active_start, color='#B0B0B0',
           alpha=0.2, width=1.0, label='Frozen (filled)')
if active_end < N_SITES:
    ax.bar(range(active_end, N_SITES), [0.0] * (N_SITES - active_end),
           color='#D0D0D0', alpha=0.2, width=1.0, label='Frozen (empty)')

# Domain wall
ax.axvline(x=WALL - 0.5, color='red', linestyle='--', linewidth=2, label='Initial domain wall')

reduction = TOTAL_QUBITS // (2 * n_active) if n_active > 0 else 0
ax.set_xlabel('Lattice Site Index', fontsize=12)
ax.set_ylabel('Total Density ⟨n↑⟩ + ⟨n↓⟩', fontsize=12)
ax.set_title(
    f'{TOTAL_QUBITS}-Qubit Fermi-Hubbard: '
    f'Scout ({scout_time:.2f}s) → CPU χ={CHI_CPU} ({sniper_time:.1f}s) → Precision χ={chi_precision} ({precision_time:.1f}s)\n'
    f'Active: {2*n_active} qubits / {TOTAL_QUBITS} total ({reduction}× reduction) | '
    f'U/t={U_INT/T_HOP:.1f}, T={T_EVOLUTION}',
    fontsize=10
)
ax.set_ylim(-0.1, 2.3)
ax.legend(loc='upper right', fontsize=9)
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## Step 7: Convergence Analysis

Let's compare the CPU and precision results to check convergence.

In [ ]:
# Convergence check
diff = [abs(c - g) for c, g in zip(cpu_density, gpu_density)]
max_diff = max(diff)
mean_diff = np.mean(diff)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 4))

# Left: density comparison
ax1.plot(global_x, cpu_density, 'o-', color='#2F847C', markersize=5, label=f'χ={CHI_CPU}')
ax1.plot(global_x, gpu_density, 's-', color='#E74C3C', markersize=5, label=f'χ={chi_precision}')
ax1.set_xlabel('Site')
ax1.set_ylabel('Density')
ax1.set_title('Density Comparison')
ax1.legend()
ax1.grid(alpha=0.3)

# Right: difference
ax2.bar(global_x, diff, color='#9B59B6', alpha=0.8)
ax2.set_xlabel('Site')
ax2.set_ylabel('|Δ density|')
ax2.set_title(f'Convergence: max diff = {max_diff:.4f}, mean = {mean_diff:.4f}')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

if max_diff < 0.01:
    print(f"✓ Excellent convergence — results are within 1%")
elif max_diff < 0.05:
    print(f"✓ Good convergence — consider higher χ for precision work")
else:
    print(f"⚠ Results differ — increase χ for the sniper tier")

## Step 8: Pipeline Timing Summary

In [ ]:
total_time = scout_time + sniper_time + precision_time

print(f"{'=' * 60}")
print(f"  PIPELINE SUMMARY")
print(f"{'=' * 60}")
print(f"  System:       {TOTAL_QUBITS} qubits ({N_SITES} sites)")
print(f"  Active:       {2 * n_active} qubits ({n_active} sites)")
print(f"  Reduction:    {TOTAL_QUBITS // (2 * n_active)}×")
print(f"{'─' * 60}")
print(f"  Tier 1 (PP Scout):       {scout_time:7.3f}s  ({scout_time/total_time*100:4.1f}%)")
print(f"  Tier 2 (MPS χ={CHI_CPU:3d}):    {sniper_time:7.3f}s  ({sniper_time/total_time*100:4.1f}%)")
print(f"  Tier 3 (MPS χ={chi_precision:3d}):   {precision_time:7.3f}s  ({precision_time/total_time*100:4.1f}%)")
print(f"{'─' * 60}")
print(f"  Total:                   {total_time:7.3f}s")
print(f"{'=' * 60}")
print(f"")
print(f"  Key insight: the MPS tiers run on {2*n_active} qubits instead of {TOTAL_QUBITS}.")
print(f"  At 1000 qubits, the same light cone means the same MPS cost — only")
print(f"  the scout grows (linearly).")

## Summary

| Tier | Backend | Qubits | Purpose | Scaling |
|------|---------|--------|---------|----------|
| 1. Scout | `PauliPropagator` | Full system | Light-cone detection | O(N) |
| 2. Sniper | `MPS` CPU χ=64 | Active only | Quick physics preview | O(n_active · χ³) |
| 3. Precision | `MPS` GPU χ=256 | Active only | Converged results | O(n_active · χ³) |

**Key takeaway:** The Lieb-Robinson bound guarantees a finite light cone. The PP scout detects it in seconds, then MPS simulates only the active region. Total cost is **independent of system size** — making 1000-qubit simulations as fast as 50-qubit ones.